In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append('..')
import os
import skimage
import time
import kaolin as kal
import tntorch as tn
import torch
import trimesh
import tqdm
import os.path as osp
import matplotlib.pyplot as plt
from t4dt.metrics import compute_metrics, hausdorff, MSDM2
from t4dt.utils import sdf2mesh
from t4dt.t4dt import reduce_tucker, qtt_stack, qtt2tensor3d, get_qtt_frame

# Constant tsdf

In [2]:
min_tsdf, max_tsdf = -0.05, 0.05
data_dir = '/scratch2/data/cape_release/'
model = '00032'
scene = 'longshort_flying_eagle'
frames = []
for frame in sorted(os.listdir(osp.join(data_dir, 'meshes', model, scene, 'posed'))):
    if frame.startswith('sdf'):
        frames.append(frame)

In [3]:
folder = osp.join(data_dir, 'meshes', model, scene, 'posed')
sdf0 = torch.load(osp.join(folder, frames[0]))
sdf1 = torch.load(osp.join(folder, frames[142]))
sdf2 = torch.load(osp.join(folder, frames[283]))

files = [(osp.join(folder, frame), osp.join(folder, frame[4:-2] + 'obj'))
            for frame in frames]

In [4]:
from typing import List
def compute_metrics_constant(
        frames: List[str],
        sdf_pred: torch.Tensor,
        min_tsdf: float,
        max_tsdf: float,
        num_sample_points: int,
        sample_frames: List[int]):
    res = torch.tensor(sdf_pred.shape)
    result = {}
    for i in tqdm.tqdm(sample_frames):
        result[i] = {}
        frame_pred = sdf_pred
        frame_pred.clamp_min_(min_tsdf)
        frame_pred.clamp_max_(max_tsdf)

        sdf_w_coords = torch.load(frames[i][0])
        sdf = sdf_w_coords['sdf']
        coords = torch.tensor(sdf_w_coords['coords'])

        tqdm.tqdm.write('Marching cube started')
        t0 = time.time()
        mesh_pred = sdf2mesh(frame_pred, coords, res)
        tqdm.tqdm.write(f'Marching cube finished. Took: {time.time() - t0} s.')

        tqdm.tqdm.write('Marching cube started')
        t0 = time.time()
        mesh_gt = sdf2mesh(sdf, coords, res)
        tqdm.tqdm.write(f'Marching cube finished. Took: {time.time() - t0} s.')

        tqdm.tqdm.write('Sampling points started')
        t0 = time.time()
        points_gt, _ = trimesh.sample.sample_surface(mesh_gt, num_sample_points)
        points_pred, _ = trimesh.sample.sample_surface(mesh_pred, num_sample_points)
        tqdm.tqdm.write(f'Sampling points finished. Took: {time.time() - t0} s.')

        points_gt = torch.tensor(points_gt[None]).cuda()
        points_pred = torch.tensor(points_pred[None]).cuda()

        chamfer_distance_error = kal.metrics.pointcloud.chamfer_distance(points_gt, points_pred)[0].detach().cpu()
        del points_gt
        del points_pred

        l2_error = torch.norm(frame_pred - sdf.clamp_min(min_tsdf).clamp_max(max_tsdf))

        tqdm.tqdm.write('Voxelgrid conversion started')
        t0 = time.time()
        vg_pred = kal.ops.conversions.trianglemeshes_to_voxelgrids(
            torch.tensor(mesh_pred.vertices[None]),
            torch.tensor(mesh_pred.faces),
            res.max().item())
        vg_gt = kal.ops.conversions.trianglemeshes_to_voxelgrids(
            torch.tensor(mesh_gt.vertices[None]),
            torch.tensor(mesh_gt.faces),
            res.max().item())
        tqdm.tqdm.write(f'Voxelgrid conversion finished. Took: {time.time() - t0} s.')

        IoU = kal.metrics.voxelgrid.iou(vg_pred, vg_gt)
        del vg_pred
        del vg_gt

        tqdm.tqdm.write('hausdorff computation started')
        t0 = time.time()
        hausdorff_dist = hausdorff(
            torch.tensor(mesh_gt.vertices), torch.tensor(mesh_gt.faces),
            torch.tensor(mesh_pred.vertices), torch.tensor(mesh_pred.faces))
        tqdm.tqdm.write(f'hausdorff computation finished. Took: {time.time() - t0} s.')

        tqdm.tqdm.write('MSDM2 computation started')
        t0 = time.time()
        MSDM2_err = MSDM2(
            torch.tensor(mesh_gt.vertices), torch.tensor(mesh_gt.faces),
            torch.tensor(mesh_pred.vertices), torch.tensor(mesh_pred.faces))
        tqdm.tqdm.write(f'MSDM2 computation finished. Took: {time.time() - t0} s.')
        result[i] = {
            'l2': l2_error,
            'chamfer_distance': chamfer_distance_error,
            'IoU': IoU[0],
            'hausdorff': hausdorff_dist,
            'MSDM2': MSDM2_err}
    return result

In [8]:
compute_metrics_constant(
    files,
    sdf0['sdf'],
    min_tsdf, max_tsdf,
    30000,
    [0, len(frames) // 2, len(frames) - 1])

  0%|                                                                                                                                                                                                                                 | 0/3 [00:00<?, ?it/s]

Marching cube started


  0%|                                                                                                                                                                                                                                 | 0/3 [00:01<?, ?it/s]

Marching cube finished. Took: 1.3789918422698975 s.
Marching cube started


  0%|                                                                                                                                                                                                                                 | 0/3 [00:03<?, ?it/s]

...

 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                        | 2/3 [09:39<04:45, 285.12s/it]

hausdorff computation finished. Took: 0.9312138557434082 s.
MSDM2 computation started


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [14:02<00:00, 280.69s/it]

Asking to calculate curvature
Asking to calculate curvature
Asking to calculate curvature
Asking to calculate curvature
Asking to calculate curvature
Asking to calculate curvature
MSDM2 computation finished. Took: 262.7808926105499 s.


{0: {'l2': tensor(0.),
  'chamfer_distance': tensor(4.2841e-05, dtype=torch.float64),
  'IoU': tensor(0.9999),
  'hausdorff': 0.0005439726856382182,
  'MSDM2': 0.004876062542257095},
 142: {'l2': tensor(12.2017),
  'chamfer_distance': tensor(0.0067, dtype=torch.float64),
  'IoU': tensor(0.0072),
  'hausdorff': 0.19801137616898737,
  'MSDM2': 0.5154686371753484},
 283: {'l2': tensor(4.1074),
  'chamfer_distance': tensor(0.0002, dtype=torch.float64),
  'IoU': tensor(0.0311),
  'hausdorff': 0.06490128288537549,
  'MSDM2': 0.3891190929040058}}

In [15]:
compute_metrics_constant(
    files,
    (sdf0['sdf'] + sdf1['sdf']) / 2,
    min_tsdf, max_tsdf,
    30000,
    [0, len(frames) // 2, len(frames) - 1])

  0%|                                                                                                                                                                                                                                 | 0/3 [00:00<?, ?it/s]

Marching cube started


  0%|                                                                                                                                                                                                                                 | 0/3 [00:01<?, ?it/s]

Marching cube finished. Took: 1.369436502456665 s.
Marching cube started


  0%|                                                                                                                                                                                                                                 | 0/3 [00:03<?, ?it/s]

...

 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                        | 2/3 [08:47<04:15, 255.89s/it]

hausdorff computation finished. Took: 1.5349617004394531 s.
MSDM2 computation started
Asking to calculate curvature
Asking to calculate curvature
Asking to calculate curvature
Asking to calculate curvature
Asking to calculate curvature
Asking to calculate curvature


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [12:57<00:00, 259.11s/it]

MSDM2 computation finished. Took: 249.64593958854675 s.


{0: {'l2': tensor(11.2982),
  'chamfer_distance': tensor(0.0062, dtype=torch.float64),
  'IoU': tensor(0.0080),
  'hausdorff': 0.1682107862003486,
  'MSDM2': 0.5376455052178554},
 142: {'l2': tensor(3.8179),
  'chamfer_distance': tensor(0.0002, dtype=torch.float64),
  'IoU': tensor(0.0412),
  'hausdorff': 0.03962373930620786,
  'MSDM2': 0.4032762050864984},
 283: {'l2': tensor(11.6279),
  'chamfer_distance': tensor(0.0068, dtype=torch.float64),
  'IoU': tensor(0.0060),
  'hausdorff': 0.1900046183604913,
  'MSDM2': 0.5454763465573684}}

# QTT

In [102]:
ckpt = torch.load('../logs/sweeps/qtt_00032_longshort_flying_eagle_high_ranks_high_thr.pt')

In [103]:
scene = ckpt[(-0.05, 0.05)][400]['compressed_scene']

In [104]:
ranks_tt = scene.ranks_tt.clone()

In [105]:
rmax = 800
ranks_tt[4::4] = torch.where(ranks_tt[4::4] < rmax, ranks_tt[4::4], rmax)
ranks_tt[3::4] = torch.where(ranks_tt[3::4] < rmax, ranks_tt[3::4], rmax)

In [106]:
lrs = tn.round_tt(scene.clone(), rmax=ranks_tt[1:-1])

In [40]:
coords = torch.tensor(
    torch.load('/scratch2/data/cape_release/meshes/00032/longshort_flying_eagle/coords.pt')['coords'])

In [107]:
frame0_qtt = get_qtt_frame(lrs, 184)
frame0 = qtt2tensor3d(frame0_qtt.torch())
mesh = sdf2mesh(frame0, coords)

obj = trimesh.exchange.obj.export_obj(mesh, include_texture=False)
with open(f'./mesh_sdf.obj', 'w') as f:
    f.write(obj)

In [51]:
scene.numcoef() / scene.numel()

tensor(0.0054, dtype=torch.float64)

In [65]:
lrs.numcoef() / lrs.numel()

tensor(0.0008, dtype=torch.float64)

In [14]:
# import numpy as np

# def get_qtt_frame(qtt_scene: tn.Tensor, frame: int, N: int = 3):
#     idxs = []
#     frame_bits = np.binary_repr(frame, qtt_scene.dim() // (N + 1))
#     bit_index = 0
#     for i in range(qtt_scene.dim()):
#         if (i + 1) % (N + 1) == 0:
#             idxs.append(int(frame_bits[bit_index]))
#             bit_index += 1
#         else:
#             idxs.append(slice(None))

#     return qtt_scene[idxs]

In [6]:
frame0_qtt = get_qtt_frame(scene, 0)
frame0 = qtt2tensor3d(frame0_qtt.torch())

In [8]:
# mesh = sdf2mesh(tuckers[284 // 2].torch(), coords, 512)
# mesh = sdf2mesh(local_res_tucker[..., 284 // 2].torch(), coords, 512)
coords = torch.tensor(
    torch.load('/scratch2/data/cape_release/meshes/00032/longshort_flying_eagle/coords.pt')['coords'])
mesh = sdf2mesh(frame0, coords)
obj = trimesh.exchange.obj.export_obj(mesh, include_texture=False)
with open('./mesh_sdf.obj', 'w') as f:
    f.write(obj)

In [ ]:
min_tsdf, max_tsdf = -0.05, 0.05
data_dir = '/scratch2/data/cape_release/'
model = '00032'
scene = 'longshort_flying_eagle'
frames = {}

for scene in sorted(os.listdir(osp.join(data_dir, 'meshes', model))
    for frame in sorted(os.listdir(osp.join(data_dir, 'meshes', model, scene, 'posed'))):
        if frame.startswith('sdf'):
            frames.append(frame)

In [75]:
frames

NameError: name 'frames' is not defined

# Stats

In [83]:
min_tsdf, max_tsdf = -0.05, 0.05
data_dir = '/scratch2/data/cape_release/'
model = '00032'
frames = {}

for scene in sorted(os.listdir(osp.join(data_dir, 'meshes', model))):
    frames[scene] = []
    for frame in sorted(os.listdir(osp.join(data_dir, 'meshes', model, scene, 'posed'))):
        if frame.startswith('watertight'):
            frames[scene].append(frame)

In [86]:
lengths = []
for s in frames.keys():
    lengths.append(len(frames[s]))

In [88]:
frames.keys()

dict_keys(['In', 'longshort_ATUsquat', 'longshort_flying_eagle', 'longshort_hips', 'longshort_pose_model', 'longshort_punching', 'longshort_shoulders_mill', 'longshort_soccer', 'longshort_squats', 'longshort_tilt_twist_left', 'shortlong_ATUsquat', 'shortlong_ballerina_spin', 'shortlong_basketball', 'shortlong_bend_back_and_front', 'shortlong_flying_eagle', 'shortlong_hips', 'shortlong_pose_model', 'shortlong_punching', 'shortlong_shoulders_mill', 'shortlong_soccer', 'shortlong_squats', 'shortlong_tilt_twist_left', 'shortshort_ATUsquat', 'shortshort_bend_back_and_front', 'shortshort_chicken_wings', 'shortshort_flying_eagle', 'shortshort_hips', 'shortshort_pose_model', 'shortshort_punching', 'shortshort_shoulders_mill', 'shortshort_soccer'])

In [49]:
scene.ranks_tt

tensor([ 1,  2,  4,  8, 16, 32, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
        50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 32, 16,  8,  4,  2,
         1])

In [54]:
a = tn.rand(scene.shape, ranks_tt=400)
a.numcoef() / a.numel()

tensor(0.0002, dtype=torch.float64)

In [51]:
scene.numcoef() / scene.numel()

tensor(1.8791e-06, dtype=torch.float64)

In [55]:
t = tn.rand([512, 512, 512, 284], ranks_tucker=50)

In [57]:
t.numcoef() / t.numel()

tensor(0.0003, dtype=torch.float64)

# Compression experiment

In [95]:
tuckers = torch.load('../logs/sweeps/tt_tucker_00032_longshort_flying_eagle_high_ranks.pt')

In [8]:
rmax = 100

[t.round_tucker(rmax=rmax) for t in tuckers]
local_res_tucker = reduce_tucker(
    [t[..., None] for t in tuckers],
    eps=1e-16, rmax=1 * rmax, algorithm='svd')

In [9]:
local_res_tucker

4D TT-Tucker tensor:

 512 512 512 284
  |   |   |   |
 100 100 100 100
 (0) (1) (2) (3)
 / \ / \ / \ / \
1   200 1000100 1

In [99]:
tuckers[-1]

3D TT-Tucker tensor:

 512 512 512
  |   |   |
 128 128 128
 (0) (1) (2)
 / \ / \ / \
1   512 128 1

In [98]:
tuckers[-1].ranks_tucker

tensor([128, 128, 128])

In [93]:
lrs.ranks_tucker

tensor([512, 512, 512, 284])

In [94]:
lrs.ranks_tt

tensor([  1, 200, 200, 100,   1])

In [ ]:
max_tt_rank = 15
low_rank_tt = local_res_tucker.clone()
low_rank_tt.round_tt(rmax=low_rank_tt.ranks_tt[1:-2].tolist() + [max_tt_rank])

In [37]:
# mesh = sdf2mesh(tuckers[284 // 2].torch(), coords, 512)
# mesh = sdf2mesh(local_res_tucker[..., 284 // 2].torch(), coords, 512)
mesh = sdf2mesh(low_rank_tt[..., 284 // 2].torch(), coords)
obj = trimesh.exchange.obj.export_obj(mesh, include_texture=False)
with open('./mesh_sdf.obj', 'w') as f:
    f.write(obj)

In [13]:
import torch
import os
import os.path as osp
import sys
import tqdm
import trimesh
import tntorch as tn
import yaml

sys.path.append(osp.join(os.path.abspath(os.getcwd()), '..',))
from t4dt.t4dt import reduce_tucker, reduce_tt, tensor3d2qtt, qtt2tensor3d, qtt_stack, get_qtt_frame
from t4dt.metrics import compute_metrics
from t4dt.utils import sdf2mesh

In [3]:
local_res = torch.load('../scripts/perda.pt')

In [29]:
torch.set_default_dtype(torch.float64)

In [44]:
f = 10
local_res_decomp = reduce_tt(
    [t[..., None] for t in local_res],
    eps=1e-8, rank=[f * 50, f * 50, 284], algorithm='eig')

In [9]:
coords = torch.tensor(
    torch.load('/scratch2/data/cape_release/meshes/00032/longshort_flying_eagle/coords.pt')['coords'])

In [45]:
mesh = sdf2mesh(local_res_decomp[..., 284 // 2].torch(), coords)
obj = trimesh.exchange.obj.export_obj(mesh, include_texture=False)
with open('./mesh_sdf.obj', 'w') as f:
    f.write(obj)

In [72]:
lrs = local_res_decomp.clone()
lrs.round_tt(rmax=[200, 200, 100], algorithm='eig')

In [73]:
mesh = sdf2mesh(lrs[..., 284 // 2].torch(), coords)
obj = trimesh.exchange.obj.export_obj(mesh, include_texture=False)
with open('./mesh_sdf.obj', 'w') as f:
    f.write(obj)

In [17]:
local_res[0]

3D TT tensor:

 512 512 512
  |   |   |
 (0) (1) (2)
 / \ / \ / \
1   50  50  1

In [74]:
lrs.numcoef() / lrs.numel()

tensor(0.0008)

In [19]:
local_res[1].ranks_tt

tensor([ 1, 50, 50,  1])

In [31]:
local_res_decomp

4D TT tensor:

 512 512 512 284
  |   |   |   |
 (0) (1) (2) (3)
 / \ / \ / \ / \
1   500 500 284 1